# Import modules

In [244]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [ ]:
            {crewDetail(crewMemberName: $crew_member_name)
            {
            movieTitle
            movieRevenue
            } 

In [317]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_crew_list = """query data ($nationality: String!, $job: String!)
{crewList(nationality: $nationality, job: $job)
{
aggregateRevenue,
name
}
}"""
query_movie_list = """query data ($crewMemberName: String!)
{movieList(crewMemberName: $crewMemberName)
{
movieTitle,
movieRevenue
}
}
"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_metadata = """query metadata {nationalityList(job:"Director"), jobList(nationality:"Arabic")}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n
queryType {\n      name\n    }\n    mutationType {\n
name\n    }\n    subscriptionType {\n      name\n    }\n
types {\n      ...FullType\n    }\n    directives {\n
name\n      description\n      locations\n      args {\n
...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n
kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n
description\n    args {\n      ...InputValue\n    }\n    type {\n
...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}"""
query_crew_list = json.dumps({'variables': {'nationality': 'Arabic', 'job': 'Art Direction'}, 'query': query_crew_list})
query_movie_list = json.dumps({'variables': {'crewMemberName': 'James Cameron'}, 'query': query_movie_list})
query_metadata = json.dumps({'query': query_metadata})
query_introspect = json.dumps({'query': query_introspect})

In [321]:
query_movie_list

'{"variables": {"crewMemberName": "James Cameron"}, "query": "query data ($crewMemberName: String!)\\n{movieList(crewMemberName: $crewMemberName)\\n{\\nmovieTitle,\\nmovieRevenue\\n}\\n}}\\n"}'

# Test handler functions

In [292]:
import handler 
importlib.reload(handler);

In [318]:
%%time
out = handler.handle(query_movie_list)

CPU times: user 523 µs, sys: 0 ns, total: 523 µs
Wall time: 532 µs


In [319]:
out

'{"data": null, "errors": ["Syntax Error GraphQL (1:107) Unexpected }\\n\\n1: query data ($crewMemberName: String!){movieList(crewMemberName: $crewMemberName){movieTitle,movieRevenue}}}\\n                                                                                                             ^\\n"]}'

In [268]:
json.loads(out)['errors'][0]

'Syntax Error GraphQL (1:219) Unexpected }\n\n1: query data ($nationality: String!, $job: String!)            {crewList(nationality: $nationality, job: $job)            {            aggregateRevenue            name            }                       }                }\n                                                                                                                                                                                                                             ^\n'

In [160]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

In [320]:
handler.schema.introspect()

OrderedDict([('__schema',
              OrderedDict([('queryType', OrderedDict([('name', 'Query')])),
                           ('mutationType', None),
                           ('subscriptionType', None),
                           ('types',
                            [OrderedDict([('kind', 'OBJECT'),
                                          ('name', 'Query'),
                                          ('description', None),
                                          ('fields',
                                           [OrderedDict([('name',
                                                          'nationalityList'),
                                                         ('description', None),
                                                         ('args',
                                                          [OrderedDict([('name',
                                                                         'job'),
                                                              

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [177]:
function_url = 'https://labs.cloud.contiamo.com/b0a002bf-d166-5ccc-a552-ebe89882cb90/v1/function/be852eaa-c701-4f73-8ed7-aceb6dd4aeb8/index_by_nationality'

In [179]:
%%time
deployed_output = requests.post(function_url,
                                data=query_data).json()

CPU times: user 25.1 ms, sys: 15.8 ms, total: 40.9 ms
Wall time: 438 ms


In [180]:
deployed_output

{'errors': ['Server returned unexpected status code: 500 - '],
 'fieldErrors': {}}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True